In [ ]:
import pandas as pd 
import pyodbc
import warnings
warnings.simplefilter('ignore')
import sqlite3

In [ ]:

import pandas as pd 
import pyodbc
import warnings
warnings.simplefilter('ignore')
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

In [ ]:
DB = {'servername': 'LAPTOPTK\SQLEXPRESS', 'database': 'DAD'}

conn_str = (
    'DRIVER={SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'Trusted_Connection=yes;'
)

conn = pyodbc.connect(conn_str)

query = "SELECT * FROM ORDERR"
order = pd.read_sql_query(query, conn)

conn.close()

In [ ]:
order['DATE_Order_date'] = pd.to_datetime(order['DATE_Order_date'])
order['Year'] = order['DATE_Order_date'].dt.year
order['Month'] = order['DATE_Order_date'].dt.month
order['Day'] = order['DATE_Order_date'].dt.day
order = order.drop('DATE_Order_date', axis=1) 

In [ ]:
conn = pyodbc.connect(conn_str)

query = "SELECT * FROM PRODUCT"
product = pd.read_sql_query(query, conn)

conn.close()

In [ ]:
conn = pyodbc.connect(conn_str)

query = "SELECT * FROM RETALER_SITE"
retailer = pd.read_sql_query(query, conn)

conn.close()

In [ ]:
order = pd.merge(order, product, left_on='PRODUCT_number', right_on='PRODUCT_id')

In [ ]:
order = pd.merge(order, retailer, left_on='RETAILER_SITE_id', right_on='RETALER_SITE_id')

In [ ]:
order

In [ ]:
order_dummy = order[['ORDER_unit_Cost', 'ORDER_Unit_price', 'ORDER_Unit_sale_price', 'PRODUCT_number', 'RETAILER_SITE_id', 'SALES_STAFF_id', 'Year','Month','Year', 'ORDER_Quantity','RETALER_SITE_CITY_name'
      ,'RETALER_SITE_REGION_name','RETALER_SITE_COUNTRYname','RETALER_SITE_TERRITORY_name','PRODUCT_TYPE_name','PRODUCT_LINE_name']]
df_encoded = pd.get_dummies(order_dummy, columns=['PRODUCT_number', 'RETAILER_SITE_id', 'SALES_STAFF_id','RETALER_SITE_CITY_name','RETALER_SITE_REGION_name','RETALER_SITE_COUNTRYname','RETALER_SITE_TERRITORY_name','PRODUCT_TYPE_name','PRODUCT_LINE_name'], drop_first=True)
df_encoded 

In [ ]:
X = df_encoded.drop('ORDER_Quantity', axis=1)  
y = df_encoded[['ORDER_Quantity']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42
)

In [ ]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

In [ ]:
y_pred = reg_model.predict(X_test)

In [ ]:
prediction_df = pd.DataFrame(y_pred, columns=['Predicted Quantity'])
y_test.reset_index(drop=True, inplace=True)
y_test_prediction_merge = pd.concat([y_test, prediction_df], axis=1)

In [ ]:
plt.scatter(y_test_prediction_merge['ORDER_Quantity'], y_test_prediction_merge['Predicted Quantity'])
plt.xlabel('Actual Quantity')
plt.ylabel('Predicted Quantity')
plt.show()

In [ ]:
mse = mean_squared_error(y_test_prediction_merge['ORDER_Quantity'], y_test_prediction_merge['Predicted Quantity'])
mae = mean_absolute_error(y_test_prediction_merge['ORDER_Quantity'], y_test_prediction_merge['Predicted Quantity'])
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')